In [2]:
import numpy as np
import pandas as pd
import json
import re
import os
from google.colab import drive
import random

In [3]:
# !pip install -q pytorch-lightning
# !pip install -q transformer
# !pip install -q huggingface_hub
!pip install -q langchain

from langchain_core.prompts import PromptTemplate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00


In [4]:
drive.mount('/content/drive/')

Mounted at /content/drive/


First we want to create instruct dataset we have output as QE input files for unaries and oxides We need to generate a variaty of initial instructions for this output.


In [5]:
unaries_dir='/content/drive/MyDrive/Fine-tuning-llms/data_unaries/'
oxides_dir='/content/drive/MyDrive/Fine-tuning-llms/data_oxides/'

list_of_unaries=os.listdir(unaries_dir)
list_of_oxides=os.listdir(oxides_dir)

In [6]:
prompt_templates=[]

prompt_templates.append(PromptTemplate.from_template("Generate an input file for Quantum Espresso single point \
                                                      energy calculations for {compound}.  Cell parameters are {cell_parameters} in angstroms\
                                                      and atomic positions are {atomic_positions} in angstroms"))
prompt_templates.append(PromptTemplate.from_template("Generate a Quantum Espresso input file for {compound} with cell parameters {cell_parameters} in angstroms\
                                                      with atomic positions {atomic_positions} in angstroms, for single-point energy evaluation."))
prompt_templates.append(PromptTemplate.from_template("Prepare an input script for Quantum Espresso to perform a single-point energy \
                                                      calculation for {compound}.  Take lattice with cell parameters {cell_parameters} in angstroms, \
                                                      and atomic positions {atomic_positions} in angstroms."))
prompt_templates.append(PromptTemplate.from_template("Write an input file for Quantum Espresso to perform a single-point energy \
                                                      calculation for {compound}.  Lattice with cell parameters {cell_parameters} in angstroms\
                                                      and atomic positions {atomic_positions} in angstroms."))


In [7]:
def extract_data_unaries(input_file: str):
  """extracts compound, cell parameters and atomic positions from input file"""
  string=input_file[input_file.find('ATOMIC_SPECIES')+len('ATOMIC_SPECIES\n'):]
  compound=string[:string.find(' ')]

  string=input_file[input_file.find('ATOMIC_POSITIONS angstrom')+len('ATOMIC_POSITIONS angstrom\n'):]
  atomic_positions=string[:string.find('K_POINTS automatic')]

  string=input_file[input_file.find('CELL_PARAMETERS angstrom')+len('CELL_PARAMETERS angstrom\n'):]
  cell_parameters=string
  return compound, cell_parameters, atomic_positions

In [8]:
data_unaries=pd.DataFrame(columns=['input','output'])
key_prompts=[]
input_files=[]
for in_file in list_of_unaries:
  with open(unaries_dir+in_file,'r') as f:
    input_file=f.read()
  compound, cell_parameters, atomic_positions=extract_data_unaries(input_file)
  prompt_template=random.choice(prompt_templates)
  key_prompts.append(prompt_template.format(compound=compound, cell_parameters=cell_parameters, atomic_positions=atomic_positions))
  input_files.append(input_file)
data_unaries['input']=key_prompts
data_unaries['output']=input_files
data_unaries.to_csv('/content/drive/MyDrive/Fine-tuning-llms/data_unaries.csv')

In [9]:
def extract_data_oxides(input_file: str):
  """extracts compound, cell parameters and atomic positions from input file"""
  string=input_file[input_file.find('CELL_PARAMETERS angstrom')+len('CELL_PARAMETERS angstrom\n'):]
  cell_parameters=string

  string=input_file[input_file.find('ATOMIC_POSITIONS angstrom')+len('ATOMIC_POSITIONS angstrom\n'):]
  atomic_positions=string[:string.find('K_POINTS automatic')]

  string=atomic_positions
  elements=[]
  while(len(string)>0):
    el=string[:string.find(' ')]
    elements.append(el)
    string=string[string.find('\n')+1:]
  compound=''
  set_elements=set(elements)
  set_elements.discard('O')
  list_elements=list(set_elements)
  list_elements.append('O')

  for el in list_elements:
    compound+=el+str(elements.count(el))

  return compound, cell_parameters, atomic_positions

In [10]:
data_oxides=pd.DataFrame(columns=['prompt','input_file'])
key_prompts=[]
input_files=[]
for in_file in list_of_oxides:
  with open(oxides_dir+in_file,'r') as f:
    input_file=f.read()
  compound, cell_parameters, atomic_positions=extract_data_oxides(input_file)
  prompt_template=random.choice(prompt_templates)
  key_prompts.append(prompt_template.format(compound=compound, cell_parameters=cell_parameters, atomic_positions=atomic_positions))
  input_files.append(input_file)
data_oxides['input']=key_prompts
data_oxides['output']=input_files
data_oxides.to_csv('/content/drive/MyDrive/Fine-tuning-llms/data_oxides.csv')

In [12]:
data=pd.concat([data_unaries,data_oxides])
data=data[['input','output']]
data.to_csv('/content/drive/MyDrive/Fine-tuning-llms/qe_input_dataset.csv')